In [1]:
from model.model import build_model
from data.process import *
import keras

2024-07-09 20:25:29.929723: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-09 20:25:29.937861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 20:25:29.949477: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 20:25:29.949510: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-09 20:25:29.957400: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_dataframe()

In [3]:

X_train, Y_train = split_input_output(train_df)
print(X_train.shape)
print(Y_train.shape)

(11389751, 11)
(11389751,)


In [4]:
X_dev, Y_dev = split_input_output(dev_df)
print(X_dev.shape)
print(Y_dev.shape)

(115047, 11)
(115047,)


## Original Dataset

In [5]:
train_dataset = convert_to_tensorflow_dataset(X_train, Y_train, 128)
dev_dataset = convert_to_tensorflow_dataset(X_dev, Y_dev, 128)

2024-07-09 20:25:42.269312: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-09 20:25:42.290619: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-09 20:25:42.290655: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-09 20:25:42.292615: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-09 20:25:42.292643: I external/local_xla/xla/stream_executor

## Oversampled Dataset

In [6]:
from data.augmentation import oversample_train_data
X_train_augmented, Y_train_augmented = oversample_train_data(train_df)
train_dataset_augmented = convert_to_tensorflow_dataset(X_train_augmented, Y_train_augmented, 128)


In [7]:
print(len(X_train_augmented.columns))
print(len(X_train.columns))

11
11


In [8]:
TRAINED = True

In [9]:
model = build_model(len(X_train.columns), 64, 1)

In [10]:

from keras import callbacks
from datetime import datetime

# make folder name of current date
folder = str(datetime.now().date())
checkpoint = callbacks.ModelCheckpoint(
    f'checkpoint/{folder}' + '/model-{epoch:02d}-{val_loss:.2f}.keras', 
    monitor='val_acc', 
    verbose=1, 
    mode='auto'
)

In [11]:
if not TRAINED:
    opt = keras.optimizers.Adam(learning_rate=0.001, weight_decay=0.01)
    model.compile(
        optimizer=opt, 
        loss='binary_crossentropy', 
        metrics=[
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
        ]
    )
    history = model.fit(
        train_dataset_augmented, 
        epochs=2, 
        verbose=1, 
        validation_data=dev_dataset, 
        callbacks=[checkpoint]
    )
else:
    model.load_weights('checkpoint/2024-07-09/model-01-1.96.keras')
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
if not TRAINED:
    df_loss_acc = pd.DataFrame(history.history)
    df_loss= df_loss_acc[['loss','val_loss']]
    df_loss.rename(columns={'loss':'train','val_loss':'validation'},inplace=True)
    df_acc= df_loss_acc[['accuracy','val_accuracy']]
    df_acc.rename(columns={'accuracy':'train','val_accuracy':'validation'},inplace=True)
    df_loss.plot(title='Model loss',figsize=(12,8)).set(xlabel='Epoch',ylabel='Loss')
    df_acc.plot(title='Model Accuracy',figsize=(12,8)).set(xlabel='Epoch',ylabel='Accuracy')

# Inference on dev dataset

In [13]:
dev_predictions = model.predict(dev_df.drop(columns=['Response']))
pd.DataFrame(dev_predictions.flatten()).to_csv('predictions.csv', index=False, header=False)


   1/3596 ━━━━━━━━━━━━━━━━━━━━ 18:38 311ms/step

I0000 00:00:1720524354.020415   67175 service.cc:145] XLA service 0x7f7ec8006000 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720524354.020458   67175 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-07-09 20:25:54.026488: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-09 20:25:54.090597: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1720524354.191216   67175 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3596/3596 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step


# Inference on test dataset

In [22]:
dev_predictions.flatten()

array([-2.1559408, -2.1560128, -2.1576216, ..., -2.117806 , -2.1144965,
       -2.1186657], dtype=float32)

In [13]:
predictions = model.predict(test_df)


I0000 00:00:1720439580.846986   17369 service.cc:145] XLA service 0x7f453c00db40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720439580.847030   17369 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-07-08 20:53:00.864954: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-08 20:53:00.940562: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


   209/239684 ━━━━━━━━━━━━━━━━━━━━ 2:53 725us/step

I0000 00:00:1720439581.240744   17369 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


239684/239684 ━━━━━━━━━━━━━━━━━━━━ 232s 966us/step


array([[-7.2552452e-06],
       [-7.2552452e-06],
       [-7.2552452e-06],
       ...,
       [-7.2552402e-06],
       [-7.2552402e-06],
       [-7.2552402e-06]], dtype=float32)